# Генерация входных данных

Подключем необходимые библиотеки. Для генерации parquet файлов там также потребуется установленная библиотека `pyarrow`  
```bash
pip install pyarrow
```

In [5]:
# Подключаем нужные нам библиотеки
import csv
import random
import pandas as pd
from datetime import datetime, timedelta

## Структура данных
Что за данные мы будем генерировать?

Таблица запросов пользователей в друзья. В этой таблице содержится идентификатор пользователя, отправившего запрос, идентификатор пользователя, получившего запрос, и дата принятия запроса.

Таблица должна иметь вид:

| Column Name    | Type    |
|----------------|---------|
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |

Пара `(requester_id, accepter_id)` - уникальный первичный ключ. Первичные ключи не должны повторяться, а также не может быть "обратных пар" (`(accepter_id, requester_id)` при существующей паре `(requester_id, accepter_id)`)

## Генерация csv
Генерировать будем безо всяких условий (то есть, у нас могут быть ошибки в дублирующихся первичных ключах и в обратных парах)

In [ ]:
num_rows = 200  # количество строк данных
num_users = 30  # количество уникальных пользователей

# Генерируем пары (requester_id, accepter_id)
pairs = set()

while len(pairs) < num_rows:
    requester = random.randint(1, num_users)
    accepter = random.randint(1, num_users)
    # Не добавляем проверок
    # if requester != accepter:
    # pair = (requester, accepter)
    # reverse_pair = (accepter, requester)
    # if pair not in pairs and reverse_pair not in pairs:
    #     pairs.add(pair)
    pair = (requester, accepter)
    pairs.add(pair)

# Генерируем случайные даты в диапазоне за 2024 год
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(num_rows)]

# Записываем в CSV файл
with open('input/requests_data_bad.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['requester_id', 'accepter_id', 'accept_date'])
    for i in range(num_rows):
        pair = pairs.pop()
        date = dates[i]
        writer.writerow([pair[0], pair[1], date.date().isoformat()])

## Генерация parquet
Теперь сгенерируем то же самое, но для parquet файла

In [ ]:
import pandas as pd

num_rows = 200  # количество строк данных

# Генерация пар (requester_id, accepter_id)
pairs = set()

while len(pairs) < num_rows:
    requester = random.randint(1, num_users)
    accepter = random.randint(1, num_users)
    pair = (requester, accepter)
    pairs.add(pair)

# Генерируем случайные даты в диапазоне за 2024 год
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(num_rows)]

# Создание DataFrame
data = {
    'requester_id': [pair[0] for pair in pairs],
    'accepter_id': [pair[1] for pair in pairs],
    'accept_date': [date.date().isoformat() for date in dates]
}

df = pd.DataFrame(data)

# Сохранение в Parquet файл
df.to_parquet('input/requests_data_bad.parquet', index=False)

## Генерация errors.txt
Генерируем файл errors.txt, в котором содержатся "ошибочные" id, которые будут удаляться из итогового набора данных

In [ ]:
errors_num = 5  # количество ошибочных id

# Генерируем список из случайных номеров в пределах от 1 до num_users
errors = random.sample(range(1, num_users), errors_num)

# Сохраняем в файл
with open('input/errors.txt', 'w') as file:
    file.write('\n'.join(map(str, errors)))